Install

In [4]:
pip install flasgger

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\L580\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [22]:
pip install flask_swagger_ui

     ---------------------------------------- 1.4/1.4 MB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\L580\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\L580\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Import Library

In [6]:
import pandas as pd
from flask import Flask, request, jsonify, make_response
import sqlite3
from flasgger import Swagger, LazyString, LazyJSONEncoder, swag_from
import re

Connect Database SQL

In [7]:
conn = sqlite3.connect('binar.db')

Define App

In [8]:

app = Flask(__name__)
app.json_encoder = LazyJSONEncoder

C:\Users\L580\AppData\Local\Temp\ipykernel_27864\1644928534.py:2: DeprecationWarning: 'app.json_encoder' is deprecated and will be removed in Flask 2.3. Customize 'app.json_provider_class' or 'app.json' instead.
  app.json_encoder = LazyJSONEncoder


Function to Clean Text

In [9]:
# lowercase
def lowercase(text):
    return text.lower()

# remove emoji
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# remove bytes character
def remove_bytes_character(text):
    return re.sub(r'[^\x00-\x7F]',' ', text)

# remove enter and unneeded chars
def remove_unneeded_char(text):
    text = re.sub('\n',' ', text)       #remove enter
    text = re.sub('user',' ', text)     #remove kata user
    text = re.sub('rt',' ', text)       #remove kata rt
    text = re.sub(r'\s+', ' ',text)     #eliminate duplicate whitespaces
    
# remove punctuation
def remove_punct(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
 
# cleansing data    
def cleansing_data(text):
    text = lowercase(text) 
    text = remove_emoji(text)
    text = remove_bytes_character(text)
    text = remove_unneeded_char(text)
    text = remove_punct(text)
    return text

# cleansing data for text
def cleansing_text(input_text):
    try: 
        output_text = cleansing_data(input_text)
        return output_text
    except:
        print("error")

Flask & Swagger

In [10]:
# call API post
@swag_from("swagger_config_post.yml", methods=['POST'])
@app.route("/text_twitter", methods=["POST"])
def twitter_text():
    request.method == "POST"
    input_text = str(request.get_json["text"])
    output_text = cleansing_data(input_text)
    sql_text = "insert into hate_speech_twitter (dirty_text,clean_text) values (?, ?)"
    result = (input_text, output_text)
    conn.execute(sql_text, result)
    conn.commit()
    print(input_text)
    print(output_text)
    return "sukses"

In [11]:
swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'docs',
            "route": '/docs.json',
            "rule_filter": lambda rule: True,
            "model_filter": lambda tag: True,
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}

swagger = Swagger(config=swagger_config)

In [12]:
# error handling
@app.errorhandler(400)
def handle_bad_request(e):
    return 'bad request!', 400

@app.errorhandler(404)
def handle_404_error(e):
    return 'not found!', 404

@app.errorhandler(500)
def handle_server_error(e):
    return 'server error', 500

In [21]:
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\L580\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
